# VHDL Implementation

In this section we will use the conifer VHDL backend to generate some model-specific hand-written VHDL for the model from part 1. We will show how to use it analagously to the HLS backend, but the main purpose is to look at the VHDL and compare with the HLS.

In [ ]:
from sklearn.datasets import make_moons
from sklearn.inspection import DecisionBoundaryDisplay
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import expit
import conifer
import json
import os
import sys

# enable more output from conifer
import logging
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
logger = logging.getLogger('conifer')
logger.setLevel('INFO')

# create a random seed at we use to make the results repeatable
seed = int('fpga_tutorial'.encode('utf-8').hex(), 16) % 2**31

In [ ]:
# INFN CNAF Setup
import os
os.environ['PATH'] = '/tools/Xilinx/Vitis_HLS/2023.2/tps/lnx64/gcc-9.3.0/bin/:' + '/tools/Xilinx/Vitis_HLS/2023.2/bin/:' + os.environ['PATH']
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2/'

## Configuration

We create our template configuration much the same as in part 1. Note that now we specify `vhdl` for the backend. We also set a different output directory.

In [ ]:
cfg = conifer.backends.vhdl.auto_config()

# print the config
print('Default Configuration\n' + '-' * 50)
print(json.dumps(cfg, indent=2))
print('-' * 50)

# modify the config
cfg['OutputDir'] = 'prj_conifer_part_1b_vhdl'
cfg['XilinxPart'] = 'xcu50-fsvh2104-2-e'

# print the config again
print('Modified Configuration\n' + '-' * 50)
print(json.dumps(cfg, indent=2))
print('-' * 50)

## Write Project
We load the model trained in part 1, applying the new configuration we defined in the previous cell, then write the VHDL project to the specified directory.

Take a look at the generated code, especially under firmware! Compare in particular `prj_conifer_part_1b_vhdl/firmware/Tree.vhd` with `Tree::decision_function` in `prj_conifer_part_1/firmware/BDT.h` which are the two implementations of the same algorithm.

The output of `tree prj_conifer_part_1b_vhdl` is below:

```
.
├── firmware
│   ├── AddReduce.vhd
│   ├── Arrays0.vhd
│   ├── BDTTestbench.vhd
│   ├── BDTTop.vhd
│   ├── BDT.vhd
│   ├── Constants.vhd
│   ├── SimulationInput.vhd
│   ├── SimulationOutput.vhd
│   ├── TestUtil.vhd
│   ├── Tree.vhd
│   └── Types.vhd
├── my_prj.json
├── SimulationInput.txt
├── synth.tcl
└── xsim_compile.sh
```


In [ ]:
conifer_model = conifer.model.load_model('prj_conifer_part_1/my_prj.json', new_config=cfg)
conifer_model.write()

## Compile

Now we compile the code for simulation, as in part 1. We need to use a VHDL simulator. This is more similar to the 'cosimulation' step that we've used with HLS than the 'c simulation' since it's clock cycle accurate. conifer supports `xsim` (Vivado built-in simulator), `modelsim`, and `ghdl`. We will use `xsim` since it is installed alongside Vivado. 

The testbench that we compile and simulate is provided by conifer. It reads data from a file (that we will write later), provides that as stimulus to the BDT module, and writes data to another file.

In [ ]:
conifer_model.compile()

## Simulate

Run the simulation to perform inference. When we call `decision_function` for this VHDL backend model, firstly the data is written to a file. Next the simulation is invoked using `xsim`, which writes the test results to another file. Then we read back the data from the file.

In [ ]:
X_test = np.load('moons_dataset/X_test.npy')
y_vhdl = conifer_model.decision_function(X_test)

## Compare

Load the model again but specifying the conifer Python backend that is useful for quick checking. Then we print out the results to hopefully see that they're similar (remember the VHDL is still using fixed-point data and the Python is using float).

In [ ]:
py_cfg = {'backend' : 'py', 'output_dir' : 'dummy', 'project_name' : 'dummy'}
conifer_py_model = conifer.model.load_model('prj_conifer_part_1/my_prj.json', new_config=py_cfg)
y_py = conifer_py_model.decision_function(X_test)

In [ ]:
y_py

In [ ]:
y_vhdl

## Build

We can synthesize the VHDL and generate a utilisation report. This synthesis will take a minute or two. Then we read the synthesis reports. Compare the LUT and FF usage from the VHDL backend model report with those from the `'vsynth'` section of the HLS model report from part 1.

In [ ]:
conifer_model.build()

In [ ]:
conifer_model.read_report()

In [ ]:
hls_model = conifer.model.load_model('prj_conifer_part_1/my_prj.json')
hls_model.read_report()